In [1]:
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary

import matplotlib as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 

init_notebook_mode(connected=True)
%matplotlib inline 

PLOT_PATH = '/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/html-graphs/'

# Loading Data 

In [2]:
ts_covid_confirmed = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
ts_covid_deaths = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
ts_covid_recovered = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

daily_report = pd.read_csv(
    '/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/data/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-15-2020.csv')

In [3]:
ts_covid_recovered.head(1)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20
0,NaN,Thailand,15.0,101.0,0,0,0,0,2,2,...,31,31,31,33,34,34,35,35,35,35


# Functions

# Preprocessing

In [4]:
def sum_china(df):
    return df.groupby(df['Country/Region'], as_index=False).sum()


def header_to_row(df):
    return df.melt(id_vars=['Country/Region', 'Lat', 'Long'],
                   var_name='Date', value_name='Value').sort_values(by='Value', ascending=False)


def top_10_countries(df):
    grouped_df = df.groupby(df['Country/Region'],
               as_index=False).sum().sort_values(by='Value', ascending=False)[:10]
    
    return grouped_df['Country/Region'].values.tolist()

## Recovered cases

In [5]:
# Transformation
transformed_covid_recov = header_to_row(sum_china(ts_covid_recovered))

# Top 10 recovered countries
countries = top_10_countries(transformed_covid_recov)

## Confirmed cases

In [6]:
# Transformation
transformed_covid_conf = header_to_row(sum_china(ts_covid_confirmed))

# Top 10 recovered countries
countries = top_10_countries(transformed_covid_conf)

# Statistics

In [7]:
transformed_covid_recov.groupby(
    transformed_covid_recov['Country/Region'], as_index=False).sum().sort_values(by='Value', ascending=False)[:10]

,Country/Region,Lat,Long,Value
28,China,59583.5185,202643.0835,1289825
68,Iran,1760.0000,2915.0000,33199
72,Italy,2365.0000,660.0000,14988
79,"Korea, South",1980.0000,7040.0000,4496
133,Spain,2200.0000,-220.0000,2289
35,Cruise Ship,1949.4035,7680.0900,2244
128,Singapore,70.5815,5710.8315,2088
74,Japan,1980.0000,7590.0000,1650
141,Thailand,825.0000,5555.0000,934
88,Malaysia,137.5000,6187.5000,649


In [8]:
transformed_covid_conf.groupby(
    transformed_covid_conf['Country/Region'], as_index=False).sum().sort_values(by='Value', ascending=False)[:10]

,Country/Region,Lat,Long,Value
28,China,59583.5185,2.026431e+05,2962462
72,Italy,2365.0000,6.600000e+02,170830
79,"Korea, South",1980.0000,7.040000e+03,121156
68,Iran,1760.0000,2.915000e+03,117303
133,Spain,2200.0000,-2.200000e+02,39259
52,Germany,2805.0000,4.950000e+02,32263
48,France,3953.1305,-2.409319e+03,31338
35,Cruise Ship,1949.4035,7.680090e+03,21260
147,US,524205.1870,-1.262350e+06,20021
74,Japan,1980.0000,7.590000e+03,10817


# Plots

## Recovered

### Top 10 Countries

In [10]:
fig = go.Figure()

for country in countries:
    recov_country = transformed_covid_recov[transformed_covid_recov['Country/Region'] == country]
    
    fig.add_trace(go.Scatter(
        x=recov_country.Date,
        y=recov_country.Value,
        name=country,
        opacity=0.8,))

# Layout update 
fig.update_layout(title_text="Top 10 recovered countries")

# Reversed xaxis
fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

plot(fig, filename=PLOT_PATH + 'Top 10 recovered countries.html', auto_open=False)

'/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/html-graphs/Top 10 recovered countries.html'

### Germany

In [11]:
fig = go.Figure()
countries = ['Germany']
for country in countries:
    recov_country = transformed_covid_recov[transformed_covid_recov['Country/Region'] == country]
    
    fig.add_trace(go.Scatter(
        x=recov_country.Date,
        y=recov_country.Value,
        name=country,
        opacity=0.8,))

# Layout update 
fig.update_layout(title_text="Recovered in Germany")

# Reversed xaxis
fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()
plot(fig, filename=PLOT_PATH + 'Recovered in Germany.html', auto_open=False)

'/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/html-graphs/Recovered in Germany.html'

## Confirmed

In [12]:
fig = go.Figure()

for country in countries:
    conf_country = transformed_covid_conf[transformed_covid_conf['Country/Region'] == country]
    
    fig.add_trace(go.Scatter(
        x=conf_country.Date,
        y=conf_country.Value,
        name=country,
        opacity=0.8,))

# Layout update 
fig.update_layout(title_text="Top 10 countries with confirmed cases")

# Reversed xaxis
fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()
plot(fig, filename=PLOT_PATH + 'Top 10 countries with confirmed cases.html', auto_open=False)

'/Users/andang/Documents/workspace/private/python/COVID-19-Dashboard/html-graphs/Top 10 countries with confirmed cases.html'